In [1]:

# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------


import os
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import McpTool, ToolSet
from dotenv import load_dotenv

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")


In [2]:
# Connect to the agents client
agents_client = AgentsClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True,
        exclude_managed_identity_credential=True
    )
)


# MCP server configuration
mcp_server_url = "https://learn.microsoft.com/api/mcp"
mcp_server_label = "mslearn"

# Initialize agent MCP tool
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
)

mcp_tool.set_approval_mode("never")

toolset = ToolSet()
toolset.add(mcp_tool)


# Create agent with MCP tool and process agent run
agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="my-mcp-agent",
    instructions="""
    You have access to an MCP server called `microsoft.docs.mcp` - this tool allows you to 
    search through Microsoft's latest official documentation. Use the available MCP tools 
    to answer questions and perform tasks."""
)


# Log info
print(f"Created agent, ID: {agent.id}")
print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")


Created agent, ID: asst_hYlrrUPpqFLUdysipONY3yIT
MCP Server: mslearn at https://learn.microsoft.com/api/mcp
Created thread, ID: thread_v8dBjTvOzyLaIrsOv6uflaFL


In [3]:
def send_prompt(agents_client, thread, prompt):
    # Create a message on the thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    # print(f"Created message, ID: {message.id}")
    

    # Create and process agent run in thread with MCP tools
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
    # print(f"Created run, ID: {run.id}")
    
    
    # # Check run status
    # print(f"Run completed with status: {run.status}")
    # if run.status == "failed":
    #     print(f"Run failed: {run.last_error}")

    # # Display run steps and tool calls
    # run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
    # for step in run_steps:
    #     print(f"Step {step['id']} status: {step['status']}")

    #     # Check if there are tool calls in the step details
    #     step_details = step.get("step_details", {})
    #     tool_calls = step_details.get("tool_calls", [])

    #     if tool_calls:
    #         # Display the MCP tool call details
    #         print("  MCP Tool calls:")
    #         for call in tool_calls:
    #             print(f"    Tool Call ID: {call.get('id')}")
    #             print(f"    Type: {call.get('type')}")
    #             print(f"    Type: {call.get('name')}")

    #     print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)


In [4]:
prompt = "Give me the Azure CLI commands to create an Azure Container App with a managed identity."
send_prompt(agents_client, thread, prompt)


Conversation:
--------------------------------------------------
ASSISTANT: To create an Azure Container App with a managed identity (either system-assigned or user-assigned) using the Azure CLI, you can use the following commands:

---

### 1. System-Assigned Managed Identity

This will create a Container App and assign a system-managed identity (Azure creates and manages the identity):

```bash
az containerapp identity assign \
  --name <APP_NAME> \
  --resource-group <RESOURCE_GROUP> \
  --system-assigned
```

Or, you can add the system identity during initial creation (from the doc, use --system-assigned):

```bash
az containerapp create \
  --name <APP_NAME> \
  --resource-group <RESOURCE_GROUP> \
  --environment <ENVIRONMENT_NAME> \
  --image <IMAGE_NAME> \
  --ingress <internal|external> \
  --target-port <PORT> \
  --system-assigned
```

---

### 2. User-Assigned Managed Identity

- First, create the managed identity:

```bash
az identity create --resource-group <RESOURCE_GROU

In [5]:
# Clean-up and delete the agent once the run is finished.
agents_client.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
